In [31]:
import gensim.models.keyedvectors as word2vec 
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import wordpunct_tokenize
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM

import numpy as np

import matplotlib.pyplot as plt
from pandas import DataFrame

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve,  roc_auc_score, classification_report
from nltk.tokenize import wordpunct_tokenize


import logging

import cx_Oracle
import pandas as pd 
from sqlalchemy import create_engine
import csv 



insertQuery=" INSERT INTO SocialData.ASTD VALUES(:1,:2)"
dsn_tns = cx_Oracle.makedsn('localhost', '1521', service_name='xe') 
conn = cx_Oracle.connect(user=r'SocialData', password='SocialData',dsn=dsn_tns,encoding = "UTF-8", nencoding = "UTF-8") 
c=conn.cursor()

#import into database ASTD.csv data

with open(u'ASTD.csv',encoding="utf-8-sig") as csvfile:
    reader = csv.reader(csvfile)
  #  next(reader)
    for row in reader:
     
        c.execute(insertQuery ,(row[0],row[1]))
        conn.commit()   
        #print(row[1])

print("Finishing importing ASTD data into database")

dsn_tns1 = cx_Oracle.makedsn('localhost', '1521', service_name='xe') #if needed, place an 'r' before any parameter in order to address any special character such as '\'.
conn1= cx_Oracle.connect(user=r'SocialData', password='SocialData', dsn=dsn_tns1,encoding = "UTF-8", nencoding = "UTF-8") #if needed, place an 'r' before any parameter in order to address any special character such as '\'. For example, if your user name contains '\', you'll need to place 'r' before the user name: user=r'User Name'


tweetsData = pd.read_sql('select * from SocialData.ASTD',conn1)
tweetsData=tweetsData[1:]
tweetsData = tweetsData.iloc[np.random.permutation(len(tweetsData))] #shuffling data
tweetsData.head()
logging.basicConfig(format='%(asctime)s : %(levelname) s : %(message)s', level=logging.INFO)

#Set random seed
#np.random.seed(24)

'''
Get Dataset
'''
#read CSV file containing tweets and labels, using Pandas , to get a dataframe
#tweetsData = pd.read_csv('ASTD.csv') #skiping these two rows as they have some bad data



#tweetsData.head()

#Dividing the dataset into features and lables
tweets = tweetsData['SENTENCE']
train_df, test_df = train_test_split(tweets, train_size=0.2)
test_df=list(test_df)
#print("Test Data: \n",test_df)
labels = tweetsData['SENTIMENT']

#print("tweets",tweets)


#check the distribution of lebels

#labels_count = labels.value_counts()
#labels_count.plot(kind="bar")
#print(labels.value_counts())

#Looks like the distribution is even

#Lower and split the dialog
#and use regular expression to keep only letters we will use nltk Regular expression package
#tkr = RegexpTokenizer('[a-zA-Z@]+')

tweets_split = []

for i, line in enumerate(tweets):
    #print(line)
    tweet = wordpunct_tokenize(line) 
    tweets_split.append(tweet)

#print("tweets_split",tweets_split[0])

'''
Use pretrained Word2Vec model 
'''

w2vModel = word2vec.KeyedVectors.load_word2vec_format('arabic-news.bin', binary=True)

#Convert words to integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets_split)
X = tokenizer.texts_to_sequences(tweets_split)
print("X[0] Before Padding",X[0])


#lenght of tweet to consider
maxlentweet = 40
#add padding
X = pad_sequences(X, maxlen=maxlentweet)
#print("X After Padding:\n",X)
print("Shape of X : " ,X.shape)
print("X[0] After Padding : ",X[0])

                              
#create a embedding layer using Google pre triained word2vec (50000 words)
embedding_layer = Embedding(input_dim=w2vModel.syn0.shape[0], output_dim=w2vModel.syn0.shape[1], weights=[w2vModel.syn0], 
                            input_length=X.shape[1])

#create model

lstm_out = 80

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(units=lstm_out))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#print("Model Summary: \n",model.summary())



#split dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, labels, test_size= 0.2)

#fit model
batch_size = 32
model.fit(X_train, Y_train, epochs=20, batch_size=batch_size)

#analyze the results
score, acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size=batch_size)
y_pred = model.predict_classes(X_test)

#print("y_pred ",y_pred)

#print("score",score)

#adding results to data base 
ResultSaving=" INSERT INTO SocialData.PREDECTIONS  VALUES(:1,:2,:3)"
y_pred2=list()

y_pred2 = [item for sublist in y_pred for item in sublist]
print("Y_prediction: \n" , y_pred2)

#print("Tweet: ",tweets[0])
#print("y_pred: ",y_pred[0][0])
#print("y_pred: ",Y_test[0][0])
Y_test=list(Y_test)
print("Y_test: \n ",Y_test)


for i in range (0,len(y_pred2)-1):
    c.execute(ResultSaving,(str(test_df[i]),str(y_pred2[i]),str(Y_test[i])))
    conn.commit()
        
print("\n Accuracy of model is \n =================== \n",acc,"\n ===================")

print("\n Finishing importing result in database")


Finishing importing ASTD data into database


C:\Users\Mona\Anaconda3\envs\tensorflow\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


X[0] Before Padding [1, 577, 404]
Shape of X :  (1589, 40)
X[0] After Padding :  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   1 577 404]


C:\Users\Mona\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:125: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


Epoch 1/20
1271/1271 [==============================] - 31s 25ms/step - loss: 0.6773 - acc: 0.5728
Epoch 2/20
1271/1271 [==============================] - 28s 22ms/step - loss: 0.4406 - acc: 0.8977
Epoch 3/20
1271/1271 [==============================] - 27s 21ms/step - loss: 0.2572 - acc: 0.9583
Epoch 4/20
1271/1271 [==============================] - 27s 21ms/step - loss: 0.1160 - acc: 0.9898
Epoch 5/20
1271/1271 [==============================] - 27s 21ms/step - loss: 0.0463 - acc: 0.9992
Epoch 6/20
1271/1271 [==============================] - 27s 21ms/step - loss: 0.0262 - acc: 0.9976
Epoch 7/20
1271/1271 [==============================] - 27s 22ms/step - loss: 0.0145 - acc: 0.9992
Epoch 8/20
1271/1271 [==============================] - 28s 22ms/step - loss: 0.0088 - acc: 0.9992
Epoch 9/20
1271/1271 [==============================] - 29s 22ms/step - loss: 0.0072 - acc: 0.9992
Epoch 10/20
1271/1271 [==============================] - 28s 22ms/step - loss: 0.0043 - acc: 1.0000
Epoch 11/